In [1]:
import cv2
import numpy as np
import pandas as pd
import os
import shutil
from tqdm import * #進度條
from keras.models import load_model

Using TensorFlow backend.


In [2]:
def check_folder_exist(path):
    if not os.path.isdir(path):
        os.mkdir(path)

In [3]:
#預測類別轉換
def label_decode(pre):
    class_path=r"D:\\yucheng\\Inception v4\\class_yellowsticky_Origin_20201203\\分類.txt"
    label=[]
    
    with open(class_path, 'r',encoding="utf-8") as f:
        for line in f.readlines():
            label.append(line.strip())
    
    return label[pre]

In [4]:
def get_imgclass():
    class_path=r"D:\\yucheng\\Inception v4\\class_yellowsticky_Origin_20201203\\分類.txt"
    classes=[]
    
    with open(class_path, 'r',encoding="utf-8") as f:
        for line in f.readlines():
            classes.append(line.strip())
    return classes

In [5]:
def get_ImgName(path):
    data=[]
    with open(path, 'r',encoding="utf-8") as f:
        for line in f.readlines():
            data.append(line.strip())
    return data
    

In [6]:
def labellist_to_One_hot(label):
    one_hot=[]
    for i in range(len(label)):
        arr=np.zeros(len(label), dtype=np.int)
        arr[i]=1
        one_hot.append(arr)
    one_hot=np.array(one_hot)  
    return one_hot
    

In [7]:
def get_one_hot_label(label):
    label_list=get_imgclass()
    one_hot=labellist_to_One_hot(label_list)
    for i in range(len(label_list)):
        if label==label_list[i]:
            return one_hot[i]
            break

In [8]:
def get_data(img_folder,ImgName_path):
    img_name=get_ImgName(ImgName_path)
    data=[]
    for i in trange(len(img_name)):
        label=None
        img=cv2.imdecode(np.fromfile(img_folder+img_name[i],dtype=np.uint8),cv2.IMREAD_COLOR) #等同=cv2.imread
        img=cv2.resize(img,(299,299))
        
        #取得img_label
        for j in range(len(img_name[i])):
            if '_' == img_name[i][j]:
                label=img_name[i][0:j]
                label=get_one_hot_label(label)
                break
        data.append([img,label])
    
    X=np.array([i[0] for i in data]).reshape(-1,299,299,3)
    Y=np.array([i[1] for i in data])
    
    return X,Y

# 計算混淆矩陣

In [ ]:
def get_confusion_matrix(X,Y,model):
    classes=get_imgclass()
    confusion_matrix=np.zeros((len(classes),len(classes)), dtype=np.int)
    pre_x=model.predict(X)
    
#     for i in range(len(pre_x)):
#         pre = np.argmax(pre_x[i])
#         real=np.argmax(Y[i])
#         confusion_matrix[pre][real]+=1

    for i in range(len(pre_x)):
        pre = np.argmax(pre_x[i])
        real=np.argmax(Y[i])
        
        if pre!=0 and real!=0:
            confusion_matrix[pre][real]+=1

    return(confusion_matrix)

In [10]:
def save_confusion_matrix(confusion_matrix,save_path='./confusion.csv'):
    classes=get_imgclass()
    data= pd.DataFrame(confusion_matrix)
    data.columns=classes
    data.index=classes
    data.to_csv(save_path, encoding='utf_8_sig')
    print('confusion_matrix儲存完成!')
    print(data.head(5))

# 計算TP、FP、FN、Precision、Recall

In [11]:
class map():
    def __init__(self,confusion_matrix):
        self.confusion_matrix=confusion_matrix
        self.result=[]
        for i in range(len(confusion_matrix)):
            TP=confusion_matrix[i][i]
            FP=0
            FN=0
            precision=0
            recall=0
            F1_score=0
                      
            #計算FP、FN
            for j in range(len(confusion_matrix[i])):
                if j!=i:
                    FN+=confusion_matrix[i][j]
                    if confusion_matrix[j][i]!=0:
                        FP+=confusion_matrix[j][i]
            #計算precision
            precision=TP/(TP+FP)
            recall=TP/(TP+FN)
             #計算F1_score
            F1_score=2*precision*recall/(precision+recall)
            
            self.result.append([TP,FP,FN,precision,recall,F1_score])
        self.result= pd.DataFrame(self.result)
        self.result.columns=['TP','FP','FN','Precision','Recall','F1_score']
        self.result.index=get_imgclass()
        
    def get(self):
        return self.result
    
    def save(self,save_path='./result.csv'):
        self.result.to_csv(save_path, encoding='utf_8_sig')
        print('result儲存完成!')
        
        
    def Precision(self):
        TP=0
        FP=0
        for i in range(len(self.result)):
            TP+=self.result['TP'][i]
            FP+=self.result['FP'][i]
        precision=TP/(TP+FP)
        return precision
    
    def Recall(self):
        TP=0
        FN=0
        for i in range(len(self.result)):
            TP+=self.result['TP'][i]
            FN+=self.result['FN'][i]
        recall=TP/(TP+FN)
        return recall
    

# 錯誤ouput

In [12]:
def output_false(X,Y,model):
    path='D:\\yucheng\\Inception v4\\class_yellowsticky_Origin_20201203\\'
    check_folder_exist(path+'false')
    img_name=get_ImgName(path+'testing.txt')
    pre_x=model.predict(X)
    
    for i in range(len(pre_x)):
        pre = np.argmax(pre_x[i])
        real=np.argmax(Y[i])
        if pre!=real:
            shutil.copyfile(path+'img\\'+img_name[i], path+'img\\copy.jpg')
            #os.rename(os.path.join(path+'img\\', img_name[i]), os.path.join(path+'img\\','copy.jpg')) 
            os.rename(os.path.join(path+'img\\', 'copy.jpg'), os.path.join(path+'false\\', 'F'+label_decode(pre)+'_'+img_name[i]))
    print('錯誤輸出完成')
    

# main()

In [13]:
if __name__ == '__main__':
    model_path=r"D:\\yucheng\\Inception v4\\class_yellowsticky_Origin_20201203\\model\\2googlenetV4_classification_ORI_20201203_shuffle.h5"
    img_folder=r'D:\\yucheng\\Inception v4\\class_yellowsticky_Origin_20201203\\img\\'
    test_ImgName_path=r"D:\\yucheng\\Inception v4\\class_yellowsticky_Origin_20201203\\testing.txt"
    save_confusion_path=r"D:\\yucheng\\Inception v4\\class_yellowsticky_Origin_20201203\\result\\confusion.csv"
    save_result_path="D:\\yucheng\\Inception v4\\class_yellowsticky_Origin_20201203\\result\\result.csv"
    
    #測試集
    X_test,Y_test=get_data(img_folder,test_ImgName_path)
    model = load_model(model_path)
    
    loss, acc = model.evaluate(X_test, Y_test, verbose=0)
    print('Test accuracy:', acc)
    print('Test loss:', loss)
    
    
    confusion_matrix=get_confusion_matrix(X_test,Y_test,model)
    print('混淆矩陣計算完成!')
    save_confusion_matrix(confusion_matrix,save_confusion_path)
    

100%|█████████████████████████████████████████████████████████████████████████████| 1143/1143 [00:01<00:00, 935.48it/s]


Test accuracy: 0.7104111980786996
Test loss: 1.9839927044649033
pre:2
real:2
pre:6
real:6
pre:2
real:2
pre:6
real:6
pre:2
real:2
pre:6
real:6
pre:6
real:6
pre:40
real:40
pre:15
real:15
pre:6
real:6
pre:1
real:28
pre:6
real:6
pre:6
real:6
pre:6
real:9
pre:1
real:1
pre:1
real:1
pre:15
real:6
pre:6
real:6
pre:26
real:7
pre:6
real:42
pre:2
real:2
pre:22
real:22
pre:38
real:38
pre:32
real:32
pre:17
real:17
pre:9
real:9
pre:14
real:14
pre:4
real:4
pre:17
real:9
pre:2
real:5
pre:6
real:6
pre:6
real:6
pre:6
real:6
pre:6
real:9
pre:6
real:6
pre:6
real:6
pre:6
real:9
pre:15
real:15
pre:6
real:6
pre:28
real:28
pre:4
real:4
pre:6
real:6
pre:28
real:26
pre:6
real:6
pre:2
real:2
pre:14
real:14
pre:6
real:6
pre:6
real:6
pre:7
real:7
pre:7
real:7
pre:6
real:6
pre:14
real:14
pre:6
real:6
pre:58
real:28
pre:6
real:6
pre:5
real:5
pre:6
real:6
pre:1
real:44
pre:9
real:9
pre:9
real:9
pre:4
real:4
pre:61
real:61
pre:15
real:15
pre:14
real:14
pre:6
real:6
pre:13
real:15
pre:9
real:6
pre:6
real:6
pre:6
real:6

pre:6
real:6
pre:1
real:13
pre:6
real:6
pre:40
real:40
pre:6
real:6
pre:15
real:15
pre:15
real:15
pre:6
real:6
pre:6
real:6
pre:6
real:6
pre:14
real:4
pre:1
real:1
pre:6
real:6
pre:8
real:8
pre:4
real:4
pre:8
real:35
pre:30
real:30
pre:6
real:6
pre:2
real:2
pre:6
real:6
pre:6
real:6
pre:1
real:35
pre:4
real:4
pre:9
real:10
pre:12
real:49
pre:15
real:15
pre:22
real:22
pre:6
real:6
pre:6
real:6
pre:2
real:2
pre:9
real:9
pre:4
real:43
pre:4
real:4
pre:6
real:6
pre:9
real:1
pre:6
real:6
pre:41
real:41
pre:2
real:2
pre:6
real:6
pre:6
real:13
pre:6
real:6
pre:15
real:15
pre:14
real:14
pre:1
real:6
pre:6
real:6
pre:17
real:17
pre:9
real:9
pre:6
real:6
pre:6
real:6
pre:9
real:9
pre:6
real:6
pre:6
real:6
pre:4
real:4
pre:44
real:44
pre:45
real:45
pre:6
real:6
pre:6
real:6
pre:46
real:2
pre:6
real:6
pre:9
real:9
pre:35
real:35
pre:6
real:6
pre:6
real:6
pre:6
real:15
pre:6
real:6
pre:1
real:1
pre:6
real:15
pre:6
real:15
pre:7
real:40
pre:14
real:14
pre:4
real:4
pre:7
real:5
pre:6
real:6
pre:9
rea

In [16]:
    map(confusion_matrix).save(save_result_path)
    map(confusion_matrix).Recall()

C:\Users\star\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in long_scalars
C:\Users\star\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in long_scalars
C:\Users\star\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars


result儲存完成!


0.7942446043165468

In [15]:
    output_false(X_test,Y_test,model)

FileExistsError: [WinError 183] 當檔案已存在時，無法建立該檔案。: 'D:\\yucheng\\Inception v4\\class_yellowsticky_Origin_20201203\\img\\copy.jpg' -> 'D:\\yucheng\\Inception v4\\class_yellowsticky_Origin_20201203\\false\\F小蜂_隱翅蟲_1_DSC_9559.jpg'